In [1]:
import numpy as np
import tensorflow as tf

In [3]:
# Set constants
HEIGHT = 100
WIDTH = 180

# Custom Weighted MSE Loss

def WeightedMSELoss(y_true: tf.Tensor, y_pred: tf.Tensor):  # in actuallity MSE might not be a good loss because of equidistance might use a sigmoid ish thing instead

    with tf.device('/CPU:0'):
      y_true_copy = tf.identity(y_true)

    y_true_list = tf.keras.backend.eval(y_true_copy)

    mutation_pos = set()  # place holder for now
    ref = y_true_list[0]  # (WIDTH, 1)
    for seq in y_true_list:
      for i, (base1, base2) in enumerate(zip(seq, ref)):
        if base1[0] != base2[0]:
          mutation_pos.add(i)
    mutation_pos = list(mutation_pos)

    # Calculate squared error
    squared_error = tf.square(y_true - y_pred)

    # generate weight
    tensor_shape = (HEIGHT, WIDTH)
    weight_tensor = tf.zeros(tensor_shape)
    for i in range(tensor_shape[0]):
        weight_tensor = tf.tensor_scatter_nd_add(weight_tensor, indices=[[i, pos] for pos in mutation_pos], updates=(tf.ones_like(mutation_pos, dtype=tf.float32)))
    weight_tensor = weight_tensor * 34
    weight_tensor = weight_tensor + 1
    weight_tensor = tf.reshape(weight_tensor, [HEIGHT, WIDTH, 1])

    # apply weights
    weighted_squared_error = squared_error * weight_tensor

    # compute mean of loss
    loss = tf.reduce_mean(weighted_squared_error)

    return loss

In [5]:
with tf.keras.utils.custom_object_scope({'WeightedMSELoss': WeightedMSELoss}):
    model = tf.keras.models.load_model(f'model1_vgg_generic_weighted_epochs_23122023', custom_objects={ 'loss': WeightedMSELoss })

ValueError: Unknown loss function: 'WeightedMSELoss'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.